<a href="https://colab.research.google.com/github/tthogho1/CompareImage/blob/main/Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch torchvision

In [ ]:
!pip install scikit-learn

In [35]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.autograd import Variable
from PIL import Image

from sklearn.metrics.pairwise import pairwise_distances
import numpy as np


In [6]:
input1 = torch.randn(100, 128)
print(input1)

tensor([[ 0.3587,  0.0720, -1.3893,  ..., -0.6334,  0.5200, -0.0132],
        [ 1.1617, -0.5860,  0.9338,  ..., -0.3169, -1.9410, -1.3910],
        [-1.4618, -0.6771, -0.8181,  ...,  1.4320, -0.7263, -0.4803],
        ...,
        [ 0.3931, -0.4351,  1.3434,  ..., -1.4457,  0.1885, -1.8502],
        [-1.1112, -0.6888, -1.6582,  ...,  1.1764, -0.0338,  0.0382],
        [ 1.8064,  0.3730,  0.0123,  ...,  1.1424, -0.6848,  0.2528]])


In [17]:
# Load the pretrained model
model = models.resnet18(pretrained=True)
# Use the model object to select the desired layer
layer = model._modules.get('avgpool')

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
!ls /content/drive/MyDrive

 1000550952.jpg   1010004899.jpg   1010220186.jpg  'Colab Notebooks'


In [18]:
# Set model to evaluation mode
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [19]:
scaler = transforms.Scale((224, 224))
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
to_tensor = transforms.ToTensor()

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:310: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  warnings.warn("The use of the transforms.Scale transform is deprecated, " +


In [20]:
def get_vector(image_name):
    # 1. Load the image with Pillow library
    img = Image.open(image_name)
    # 2. Create a PyTorch Variable with the transformed image
    t_img = Variable(normalize(to_tensor(scaler(img))).unsqueeze(0))
    # 3. Create a vector of zeros that will hold our feature vector
    #    The 'avgpool' layer has an output size of 512
    my_embedding = torch.zeros(512)
    # 4. Define a function that will copy the output of a layer
    def copy_data(m, i, o):
        my_embedding.copy_(o.data.reshape(o.data.size(1)))
    # 5. Attach that function to our selected layer
    h = layer.register_forward_hook(copy_data)
    # 6. Run the model on our transformed image
    model(t_img)
    # 7. Detach our copy function from the layer
    h.remove()
    # 8. Return the feature vector
    return my_embedding


In [13]:
!ls /content/drive/MyDrive

 1000550952.jpg   1010004899.jpg   1010220186.jpg  'Colab Notebooks'


In [21]:
pic_one_vector = get_vector("/content/drive/MyDrive/1010004899.jpg")
pic_two_vector = get_vector('/content/drive/MyDrive/1010220186.jpg')

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [22]:
# Using PyTorch Cosine Similarity
cos = nn.CosineSimilarity(dim=1, eps=1e-6)
cos_sim = cos(pic_one_vector.unsqueeze(0),
              pic_two_vector.unsqueeze(0))
print('\nCosine similarity: {0}\n'.format(cos_sim))


Cosine similarity: tensor([0.6713])



In [31]:
x_list = []
x_list.append(pic_one_vector)
x_list.append(pic_two_vector)
print(torch.stack(x_list))
torch.stack(x_list).size()
print(pic_one_vector)

tensor([[1.4589, 0.3039, 0.3280,  ..., 0.9439, 3.5692, 0.4273],
        [0.4054, 1.7750, 1.9254,  ..., 0.8445, 0.5413, 0.0639]])
tensor([1.4589e+00, 3.0386e-01, 3.2796e-01, 1.9138e-01, 2.8005e-01, 1.3872e-01,
        3.6402e-01, 1.2477e-01, 7.3426e-01, 1.5970e-01, 4.4163e-01, 5.0515e-01,
        8.2809e-01, 2.1711e-01, 3.8365e-02, 1.5729e-01, 2.1684e-01, 0.0000e+00,
        1.7634e-01, 7.2019e-01, 2.1628e-02, 2.8407e+00, 1.0263e-01, 2.1309e-01,
        1.2879e-01, 6.5316e-01, 4.1290e-03, 7.3910e-01, 3.4384e-01, 2.1728e-01,
        4.6472e-01, 5.6511e-01, 1.2451e+00, 5.8634e-02, 4.3798e-01, 5.4457e-01,
        5.3542e-01, 5.9974e-01, 3.3257e+00, 1.8086e-01, 5.2549e-01, 3.4851e-01,
        7.4715e-02, 8.0584e-01, 1.1847e+00, 3.5332e-01, 1.9594e+00, 1.6478e-03,
        1.5889e+00, 8.9115e-02, 2.6879e+00, 1.3961e+00, 1.4061e+00, 3.9708e-01,
        2.2176e-02, 7.0869e-01, 1.0490e+00, 4.5120e-01, 7.8677e-01, 7.1630e-01,
        0.0000e+00, 2.6368e-01, 1.1048e+00, 1.5186e-01, 1.7270e+00, 1.3

In [37]:
similarity_matrix = 1 - pairwise_distances(torch.stack(x_list), metric='cosine')
np.fill_diagonal(similarity_matrix, 0)
print(similarity_matrix.shape)


(2, 2)
